In [1]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv('.secrets')

/Users/vicente/Library/Caches/pypoetry/virtualenvs/gen-ai-demos-umKuaqgj-py3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


True

In [2]:
class BoardGameResearchPlanner(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query when researching board games")
    subquestions: list[str] = dspy.OutputField(desc="A list of 5 or more subquestions that dive deeper in the theme the user is researching.")

class BoardGameResearcher(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestion: str = dspy.InputField(desc="The subquestion you are doing deep research on")
    context = dspy.OutputField(desc="The context learned by answering the subquestion while relevant to the initial question")

class BoardGameQuestionAnswer(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query about board games")
    answer = dspy.OutputField(desc="The final answer including all the context researched about board games.")

class BoardGameResearchSummarizer(dspy.Signature):
    """Condense retrieved information into a focused summary."""
    question = dspy.InputField(desc="The user's initial query about board games")
    context: list[str] = dspy.InputField(desc="Research context discovered about the board games you are about to recommend")
    summary = dspy.OutputField(desc="Condense retrieved information into a focused summary relevant to the provided context and user question. Provide specific details supporting the answer to the question.")


class BoardGameAssistant(dspy.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.plan_research = dspy.ChainOfThought(BoardGameResearchPlanner)
        self.research = dspy.ChainOfThought(BoardGameResearcher)
        self.summarize = dspy.ChainOfThought(BoardGameResearchSummarizer)
        self.answer = dspy.ChainOfThought(BoardGameQuestionAnswer)

    def forward(self, question):
        subquestions = self.plan_research(question=question).subquestions
        research_context = []
        print(f"Question: {question} research tasks: {len(subquestions)}")
        for subquestion in subquestions:
            print(f" - Researching: {subquestion}")
            research = self.research(subquestion=subquestion, question=question)
            research_context = research_context + [research.context]

        final_research = self.summarize(context=research_context, question=question)

        return dspy.Prediction(answer=final_research.summary)

In [3]:
lm = dspy.LM('openai/gpt-4o', api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=lm)

In [4]:
assistant = BoardGameAssistant()
recommendation = assistant(question="What is the best opening move in Catan?")

Question: What is the best opening move in Catan? research tasks: 7
 - Researching: What are the most important resources to prioritize in the opening move of Catan?
 - Researching: How does the number of players affect the strategy for the opening move in Catan?
 - Researching: What are the advantages and disadvantages of starting near ports in Catan?
 - Researching: How should one balance between resource diversity and production frequency in the opening move?
 - Researching: What are common mistakes to avoid when choosing the initial settlement locations in Catan?
 - Researching: How can the initial placement strategy change based on the specific layout of the board?
 - Researching: What role does trading play in determining the best opening move in Catan?


In [5]:
recommendation


Prediction(
    answer='The best opening move in Catan involves placing initial settlements to maximize access to key resources like brick, wood, and wheat, ensuring a balanced mix and high-frequency numbers for steady resource flow. Strategy should adapt based on the number of players, with a focus on resource balance in 4-player games. Consideration of ports for trading advantages, while balancing resource diversity, is crucial. Understanding the board layout and potential trading opportunities enhances strategic positioning, avoiding common mistakes like resource imbalance and poor expansion potential.'
)

In [6]:
lm.inspect_history(n=10)





[2025-02-03T12:16:27.636671]

System message:

Your input fields are:
1. `question` (str): The user's initial query when researching board games

Your output fields are:
1. `reasoning` (str)
2. `subquestions` (list[str]): A list of 5 or more subquestions that dive deeper in the theme the user is researching.

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## subquestions ## ]]
{subquestions}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `subquestions`.


User message:

[[ ## question ## ]]
What is the best opening move in Catan?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## sub